In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
from collections import Counter
from nltk import ngrams
from utils import get_tokenized_sentences

In [67]:
class WordCounter:
    def __init__(self, sentence_generator):
        self.sentence_generator = sentence_generator
        self.sentence_count = 0
        self.token_count = 0
        self.all_ngram_counts = {}
        for ngram_length in range(1, 6):
            self.all_ngram_counts[ngram_length] = {}
        
    def count(self):
        for sentence in self.sentence_generator:
            if sentence:
                self.sentence_count += 1
            for token in sentence:
                self.token_count += 1
            for ngram_length in range(1, 6):
                ngram_counts = self.all_ngram_counts[ngram_length]
                for i, sentence_ngram in enumerate(ngrams(sentence, ngram_length)):
                    ngram_count = ngram_counts.setdefault(sentence_ngram, {'start': 0, 'all': 0})
                    if i == 0:
                        ngram_count['start'] += 1
                    ngram_count['all'] += 1

In [68]:
trainW_generator = get_tokenized_sentences('data/trainW_token_end.txt')
trainW_counter = WordCounter(trainW_generator)
trainW_counter.count()

In [74]:
trainT_generator = get_tokenized_sentences('data/trainT_token_end.txt')
trainT_counter = WordCounter(trainT_generator)
trainT_counter.count()

In [77]:
test1_generator = get_tokenized_sentences('data/test1_token_end.txt')
test1_counter = WordCounter(test1_generator)
test1_counter.count()

In [78]:
test2_generator = get_tokenized_sentences('data/test2_token_end.txt')
test2_counter = WordCounter(test2_generator)
test2_counter.count()